In [1]:
import numpy

import scipy.special

import matplotlib.pyplot
%matplotlib inline

In [2]:
class neuralNetwork:
        
        #Inicializar a rede neural
        def __init__(self, inputNodes, hiddenNodes, outputNodes, learningRate):
            self.inodes = inputNodes
            self.hnodes = hiddenNodes
            self.onodes = outputNodes
            
            #Criando o peso das ligações com o 1/raiz(num of incoming links) com uma distribuição normal
            self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
            self.who = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.onodes, self.hnodes))
            
            self.lr = learningRate
            
            #Inicializando a função de sigmoid, a função de ativação
            self.activation_function = lambda x: scipy.special.expit(x)
            
            pass
        
        #Treinar a rede neural
        def train(self, inputs_list, targets_list):
            inputs = numpy.array(inputs_list, ndmin=2).T
            targets = numpy.array(targets_list, ndmin = 2).T
             
            hidden_inputs = numpy.dot(self.wih, inputs)
            
            hidden_outputs = self.activation_function(hidden_inputs)
            
            final_inputs = numpy.dot(self.who, hidden_outputs)
            
            final_outputs = self.activation_function(final_inputs)
            #-----------#
            
            targets = numpy.array(targets_list, ndmin=2).T
            
            #error is the (target - actual)
            output_errors = targets - final_outputs
            
            #hidden layer error
            hidden_errors = numpy.dot(self.who.T, output_errors)
            
            #atualizar os pesos entre hidden e output layers
            self.who += self.lr * numpy.dot((output_errors * final_outputs * (1 - final_outputs)), numpy.transpose(hidden_outputs))
            
            #atualizar os pesos entre hidden e input layers
            self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)), numpy.transpose(inputs))

            
            pass
        
        #Dar uma resposta, dada uma entrada
        def query(self, inputs_list):
            #Converter inputs list para array 2d, esse T é de transposta
            inputs = numpy.array(inputs_list, ndmin=2).T
            
            #Multiplicar a matriz de peso entre input e hidden com os inputs, para ter os inputs do hidden
            hidden_inputs = numpy.dot(self.wih, inputs)
            
            #Calculando os sinais de saida da função de ativação
            hidden_outputs = self.activation_function(hidden_inputs)
            
            final_inputs = numpy.dot(self.who, hidden_outputs)
            
            final_outputs = self.activation_function(final_inputs)
            
            return final_outputs

In [3]:
inputNodes = 784
hiddenNodes = 100
outputNodes = 10

learningRate = 0.1

n = neuralNetwork(inputNodes, hiddenNodes, outputNodes, learningRate)

In [6]:
training_data_file = open('mnist_dataset/mnist_train.csv', 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [7]:
#Treinar a rede neural
epochs = 1
for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(outputNodes)+ 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [8]:
#Teste
test_data_file = open('mnist_dataset/mnist_test.csv', 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [9]:
#Testar e analisar

scorecard = []

for record in test_data_list:
    all_values = record.split(',')
    
    correct_label = int(all_values[0])
    #print(correct_label, 'correct_label')
    
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    
    label = numpy.argmax(outputs)
    #print(label, 'Network`s answer')
    
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
    
    pass
#print(scorecard)
scorecard_array = numpy.asarray(scorecard)
print ('Perfomance = ', scorecard_array.sum() / scorecard_array.size)

Perfomance =  0.9494


In [67]:
#0.9445 with 1 epcoh and 0.2 learning rate
#0.9611 with 5 epoch and 0.2 learning rate
#0.9660 with 7 epoch and 0.1 learning rate